In [ ]:
import sys
sys.path.append("/home/ubuntu/mlab/days/w3d3/rl_env/lib/python3.8/site-packages")
sys.path.append("/home/ubuntu/mlab/days/w3d3/rl_env/lib/python3.9/site-packages")

In [ ]:
import torch
from torch import nn
import rl_tests

In [ ]:
import gym
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
from IPython.display import Video
from video_recorder import VideoRecorder

In [ ]:
env_name = "CartPole-v1"
record = True
video_name = "test_0"

env = gym.make(env_name)
state = env.reset()
done = False
total_reward = 0

def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.axis('off')
    display.clear_output(wait=True)
    display.display(plt.gcf())

states = 0
# Ensure that VideoRecorder is initialized outside
video_recorder = VideoRecorder(env,
    f"videos/{video_name}.mp4"
)    
while not done:
    states += 1
    if record:
        video_recorder.capture_frame()
    else:  
        show_state(env)
    state, reward, done, _ = env.step(env.action_space.sample()) # Take a random action
    # print(f"{reward=}")
    total_reward += reward

print(f"total reward: {total_reward}")
print(f"num steps taken: {states}")
if record:
    video_recorder.close()


INFO: Making new env: CartPole-v1
INFO: Starting new video recorder writing to videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Starting ffmpeg with "ffmpeg -nostats -loglevel error -y -f rawvideo -s:v 600x400 -pix_fmt rgb24 -framerate 50 -i - -vf scale=trunc(iw/2)*2:trunc(ih/2)*2 -vcodec libx264 -pix_fmt yuv420p -r 50 videos/test_0.mp4"
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0.mp4
DEBUG: Capturing video frame: path=videos/test_0

In [ ]:
dir(gym.logger)

['DEBUG',
 'DISABLED',
 'ERROR',
 'INFO',
 'MIN_LEVEL',
 'WARN',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'colorize',
 'debug',
 'error',
 'info',
 'setLevel',
 'set_level',
 'warn',
 'warnings']

In [ ]:
gym.logger.set_level(gym.logger.ERROR)

In [ ]:
import os
os.listdir(os.getcwd())

['dqn_pg.ipynb',
 '.ipynb_checkpoints',
 'basic_rl_template.ipynb',
 'videos',
 '__pycache__',
 'breakout.mp4',
 'rl_tests.py',
 'video_recorder.py',
 'earth.mp4',
 'rl_env',
 'solution']

In [ ]:
# Display a saved video 
Video("videos/test_0.mp4")


In [ ]:
class QNetwork(torch.nn.Module):
    def __init__(self, in_size: int, hidden_size: int, out_size: int) -> None:
        super().__init__()
        self.mlp = torch.nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, out_size),
        )
    def forward(self, obs: torch.Tensor) -> torch.Tensor:
        """
        Possibly in a batched manner:
            Takes an observation vector (e.g. flattened image),
            outputs a vector of rewards for each action
        """
        return self.mlp(obs)
    
rl_tests.test_q_net(QNetwork)

test_q_net MATCH!!!!!!!!
 SHAPE (16, 23) MEAN: 0.01303 STD: 0.1371 VALS [0.04156 -0.1465 -0.1255 0.186 -0.0694 -0.03668 0.01025 -0.07389 -0.2283 0.3127...]


In [ ]:
import torch

def make_choice(env: gym.Env, eps: float, net: nn.Module, 
                obs: torch.Tensor, device: str
            ) -> int:
    batch_size = obs.shape[0]
    # rewards_for_each_action shape: batch_size, num_actions

    rewards_for_each_action = net(obs)
    actions = torch.where(torch.rand(batch_size) < eps, 
                    # TODO: Make this batches
                    torch.tensor([env.action_space.sample() for _ in range(batch_size)], dtype=torch.long), 
                    torch.argmax(rewards_for_each_action, dim=-1))
    # print(f"{obs=}")
    # print(f"{actions=}")
    return actions.to(device)

In [ ]:
qnetwork.mlp[0].weight.dtype

torch.float32

In [ ]:
torch.float

torch.float32

In [ ]:
sys.getsizeof(float)

416

In [ ]:
def evaluate(model: nn.Module, env: gym.Env, eps: float = 0.05, video_name = "test_1") -> float:
    
    state = env.reset()
    done = False
    total_reward = 0

    states = 0
    # Ensure that VideoRecorder is initialized outside
    video_recorder = VideoRecorder(env,
        f"videos/{video_name}.mp4"
    )    
    with torch.no_grad():
        while not done:
            states += 1
            if record:
                video_recorder.capture_frame()
            else:  
                show_state(env)
            # Batch the observation
            obs = torch.tensor(state, dtype=torch.float).unsqueeze(0)
            state, reward, done, _ = env.step(make_choice(env, eps, 
                                                          model, obs, 'cuda:0').item())
            # print(f"{reward=}")
            total_reward += reward

        print(f"total reward: {total_reward}")
        print(f"num steps taken: {states}")
        if record:
            video_recorder.close()
    return total_reward


In [ ]:
    """
    https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py
    Description:
        A pole is attached by an un-actuated joint to a cart, which moves along
        a frictionless track. The pendulum starts upright, and the goal is to
        prevent it from falling over by increasing and reducing the cart's
        velocity.
    Source:
        This environment corresponds to the version of the cart-pole problem
        described by Barto, Sutton, and Anderson
    Observation:
        Type: Box(4)
        Num     Observation               Min                     Max
        0       Cart Position             -2.4                    2.4
        1       Cart Velocity             -Inf                    Inf
        2       Pole Angle                -0.209 rad (-12 deg)    0.209 rad (12 deg)
        3       Pole Angular Velocity     -Inf                    Inf
    Actions:
        Type: Discrete(2)
        Num   Action
        0     Push cart to the left
        1     Push cart to the right
        Note: The amount the velocity that is reduced or increased is not
        fixed; it depends on the angle the pole is pointing. This is because
        the center of gravity of the pole increases the amount of energy needed
        to move the cart underneath it
    Reward:
        Reward is 1 for every step taken, including the termination step
    Starting State:
        All observations are assigned a uniform random value in [-0.05..0.05]
    Episode Termination:
        Pole Angle is more than 12 degrees.
        Cart Position is more than 2.4 (center of the cart reaches the edge of
        the display).
        Episode length is greater than 200.
        Solved Requirements:
        Considered solved when the average return is greater than or equal to
        195.0 over 100 consecutive trials.
    """

"\nhttps://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py\nDescription:\n    A pole is attached by an un-actuated joint to a cart, which moves along\n    a frictionless track. The pendulum starts upright, and the goal is to\n    prevent it from falling over by increasing and reducing the cart's\n    velocity.\nSource:\n    This environment corresponds to the version of the cart-pole problem\n    described by Barto, Sutton, and Anderson\nObservation:\n    Type: Box(4)\n    Num     Observation               Min                     Max\n    0       Cart Position             -2.4                    2.4\n    1       Cart Velocity             -Inf                    Inf\n    2       Pole Angle                -0.209 rad (-12 deg)    0.209 rad (12 deg)\n    3       Pole Angular Velocity     -Inf                    Inf\nActions:\n    Type: Discrete(2)\n    Num   Action\n    0     Push cart to the left\n    1     Push cart to the right\n    Note: The amount the velocity t

In [ ]:
import random


ValueError: empty range for randrange() (35, 5, -30)

In [ ]:
import copy

In [ ]:
from collections import deque

num_observations = env.observation_space.shape[0]
num_actions = env.action_space.n
qnetwork = QNetwork(num_observations, 64, num_actions)

def train(q: nn.Module, env: gym.Env, episodes: int = 1000, eps: float = 0.05, max_buffer_size = 10000, discount = 0.98,
          train_freq = 16, batch_size = 128, lr = 1e-3, steps = 20000, video_name = "train_0") -> float:
    
    # (observation, action, reward, done)
    # We can index into next_observation: if current observation is at index i then the next observation is at index i+1
    buffer = deque()
    
    num_steps_taken = 0
    check_for_training = 10000

    frozen_q = copy.deepcopy(q)
    frozen_q.eval()
    q.train()
    optimizer = torch.optim.Adam(q.parameters(), lr=lr)
    
    # Ensure that VideoRecorder is initialized outside
    video_recorder = VideoRecorder(env,
        f"videos/{video_name}.mp4"
    )    

    for episode in range(episodes):
        print('test')
        cur_state = env.reset()
        done = False
        episode_reward = 0

        while not done:
            num_steps_taken += 1
            if record:
                video_recorder.capture_frame()
            else:  
                show_state(env)
            # Batch the observation
            obs = torch.tensor(state, dtype=torch.float).unsqueeze(0)
            action = make_choice(env, eps, q, obs, 'cuda:0').item()
            new_state, reward, done, _ = env.step(action)
            
            # If buffer is full, take oldest time out
            if len(buffer) == max_buffer_size:
                buffer.popleft()
            buffer.append((cur_state, action, reward, done)) 
                
            new_state = cur_state
            # print(f"{reward=}")
            episode_reward += reward

                
            if num_steps_taken % train_freq == 0 and num_steps_taken > batch_size:
                train_freq += 1
                # Get sample indices
                # Doing a len(buffer) - 1 so there's always a next sample
                buffer_size = min(len(buffer), max_buffer_size)
                # print(f"{buffer_size=}")
                # print(f"{batch_size=}")
                buffer_indices = random.sample(range(buffer_size - 1), batch_size)
                optimizer.zero_grad()
                loss = 0
                for index in buffer_indices:
                    
                    cur_state, cur_action, cur_reward, cur_done = buffer[index]
                    next_state, _, _, next_state_done = buffer[index+1]
                    
                    cur_state = torch.tensor(cur_state, dtype=torch.float)
                    next_state = torch.tensor(next_state, dtype=torch.float)
                    
                    with torch.no_grad():
                        # If we have no next state, then we can't do a max over the Q function
                        # So we resort to taking the reward
                        if cur_done:
                            target = reward
                        else:
                            target = reward + discount * max(frozen_q(next_state))
                    
                    predicted = q(cur_state)[cur_action]
                    
                    loss += (target - predicted)**2
                    
                loss /= batch_size
                loss.backward()
                optimizer.step()
                                    
                # Update frozen Q network
                frozen_q.load_state_dict(q.state_dict())
                # print(f"{q.mlp[0].weight[0,0].item()=}")
                # print(f"{frozen_q.mlp[0].weight[0,0].item()=}")

        if episode % 1000 == 0:
            print(f"episode {epi reward: {episode_reward}")
        if num_steps_taken >= steps:
            break
        if num_steps_taken % check_for_training == 0:
            print("evaluating agent")
            evaluate(frozen_q, env, eps=0, video_name=f"training_checkpoint_{num_steps_taken}")
        print(f"num steps taken: {num_steps_taken}")
    if record:
        video_recorder.close()
train(qnetwork, env)

episode reward: 10.0
num steps taken: 10
episode reward: 10.0
num steps taken: 20
episode reward: 10.0
num steps taken: 30
episode reward: 10.0
num steps taken: 40
episode reward: 10.0
num steps taken: 50
episode reward: 10.0
num steps taken: 60
episode reward: 8.0
num steps taken: 68
episode reward: 10.0
num steps taken: 78
episode reward: 10.0
num steps taken: 88
episode reward: 10.0
num steps taken: 98
episode reward: 8.0
num steps taken: 106
episode reward: 10.0
num steps taken: 116
episode reward: 10.0
num steps taken: 126
episode reward: 9.0
num steps taken: 135
q.mlp[0].weight[0,0].item()=0.2603098750114441
frozen_q.mlp[0].weight[0,0].item()=0.2603098750114441
episode reward: 11.0
num steps taken: 146
q.mlp[0].weight[0,0].item()=0.26082485914230347
frozen_q.mlp[0].weight[0,0].item()=0.26082485914230347
episode reward: 10.0
num steps taken: 156


<ipython-input-118-5d26bf5aaa2c>:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cur_state = torch.tensor(cur_state, dtype=torch.float)
<ipython-input-118-5d26bf5aaa2c>:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_state = torch.tensor(next_state, dtype=torch.float)


q.mlp[0].weight[0,0].item()=0.26045557856559753
frozen_q.mlp[0].weight[0,0].item()=0.26045557856559753
episode reward: 10.0
num steps taken: 166
q.mlp[0].weight[0,0].item()=0.2599823772907257
frozen_q.mlp[0].weight[0,0].item()=0.2599823772907257
episode reward: 9.0
num steps taken: 175
q.mlp[0].weight[0,0].item()=0.2595096528530121
frozen_q.mlp[0].weight[0,0].item()=0.2595096528530121
episode reward: 10.0
num steps taken: 185
q.mlp[0].weight[0,0].item()=0.2590581178665161
frozen_q.mlp[0].weight[0,0].item()=0.2590581178665161
episode reward: 11.0
num steps taken: 196
q.mlp[0].weight[0,0].item()=0.25868338346481323
frozen_q.mlp[0].weight[0,0].item()=0.25868338346481323
episode reward: 9.0
num steps taken: 205
q.mlp[0].weight[0,0].item()=0.25836995244026184
frozen_q.mlp[0].weight[0,0].item()=0.25836995244026184
episode reward: 10.0
num steps taken: 215
q.mlp[0].weight[0,0].item()=0.25808462500572205
frozen_q.mlp[0].weight[0,0].item()=0.25808462500572205
q.mlp[0].weight[0,0].item()=0.25791

In [ ]:
qnetwork.mlp[0].weight[0,0]

tensor(-0.1145, grad_fn=<SelectBackward0>)

In [ ]:
# Let's train the QNetwork from the buffer
buffer_indices  = random.random_indices(num_samples, (0, buf_size-1))
obs, action, reward, done = buffer[i]
next_obs, _, _, _ = buffer[i+1]

deque([])